# No, Python is Not Too Slow for Computational Economics

### John Stachurski

#### Australian National University

Acknowledgements: Thanks to anonymous friends for helpful comments

Date: September 2018

## Introduction

In [this paper](https://github.com/jstac/julia_python_comparison/blob/master/Update_March_23_2018.pdf), S. Boragan Aruoba and Jesus Fernandez-Villaverde (AFV) study the relative speed of a range of programming lanugages by testing them on a value function iteration routine.  In the abstract they state:

**"The central conclusions of our original paper remain unaltered: C++ is the fastest alternative, Julia offers a great balance of speed and ease of use, and Python is too slow."**

Even the authors' own findings do not support this conclusion.  In their table 1, Python combined with its scientific libraries is actually reported as slightly faster than Julia (2.31 seconds for Python vs 2.35 for Julia).

We rerun the code below and find a similar outcome: The Python code runs in around 3.5 seconds, compared to Julia's 4.5.  The execution environment is 

```
Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               142
Model name:          Intel(R) Core(TM) i5-7300U CPU @ 2.60GHz
```

We are using the latest version of each language and scientific ecosystem at the time of writing: Julia 1.0 and Anaconda Python 5.2.

Of course, we are comparing only Julia vs Python plus its scientific libraries (as opposed to Julia vs pure Python), since no scientist would consider foregoing the scientific libraries when implementing computationally intensive algorithms in Python.



## Additional Comments

1. Python is massively popular for both general purpose and scientific computing (see, e.g., [this ranking](https://spectrum.ieee.org/static/interactive-the-top-programming-languages-2018)).  Its popularity is partly due to (and partly caused by) the huge range of third party libraries.  Within the scientific domain, these libraries include high quality array processing software and a powerful JIT compiler.  

2. Python's primary scientific JIT compiler (Numba) and Julia's JIT compiler are built on the same architecture (LLVM).  Similar execution times are to be expected.  From this perspective it is surprising that the Python code runs faster on this machine.  Other users are likely to get slightly different relative run times.

3. AFV do not display any profiling of their codes in the different languages. No memory footprint, no flop rates, etc. are shown. This matters because memory access patterns can always be optimized, especially in small problems as treated here.  One would have to take into account all the individual features of the programming languages to make a fair comparison. (Most likely there would be no surprise, as their performance behavior is known.)  A useful refernence is: S. Goedecker and A. Hoisie. Performance Optimization of Numerically Intensive Codes (SIAM), 2001. ISBN 978-0898714845. 

4. As an aside, the author thinks Julia is a great project and hopes that it succeeds.  Python wouldn't be where it is now without the flow of ideas (and competitive pressure) from Julia.


## Timings

The code below is exactly as in the [GitHub](https://github.com/jesusfv/Comparison-Programming-Languages-Economics) repo, except for the following changes, consistent comments made in the 2018 paper. 

1. Doubled the length of the capital grid.
2. Convergence tolerance is 1e-8.
3. Print out status after every 20 iterations.
4. Changed print statements to print functions (Python 3 compliance)

In [22]:
import numpy as np
import math
import time
from numba import njit, prange

In [23]:
β = 0.95
α = 1/3
A = np.array([0.9792, 0.9896, 1.0000, 1.0106, 1.0212])  # Productivity

# Transition matrix
π = np.array([[0.9727, 0.0273, 0.0000, 0.0000, 0.0000],
              [0.0041, 0.9806, 0.0153, 0.0000, 0.0000],
              [0.0000, 0.0082, 0.9837, 0.0082, 0.0000],
              [0.0000, 0.0000, 0.0153, 0.9806, 0.0041],
              [0.0000, 0.0000, 0.0000, 0.0273, 0.9727]])

# Steady state values

k_star = (α * β)**(1 / (1 - α))  # Steady state capital
y_star = k_star**α               # Steady state output
c_star = y_star - k_star         # Steady state consumption

# Set up capital grid
n = 35640                        # Number of grid points
k_grid = np.linspace(0.5 * k_star, 1.5 * k_star, n)

In [50]:
@njit(parallel=True)
def innerloop(k_grid, v, A, π, β):
    
    E_v = v @ π.T                    # Expected value function
    y_grid = np.outer(k_grid**α, A)  # Output grid
    
    v_new = np.empty_like(y_grid)
    σ = np.empty_like(y_grid)
    k_next = 0
    
    for a in prange(len(A)):
        
        for i in prange(len(k_grid)):

            v_max = -1e5
            k_prime = k_grid[0]

            for j in prange(k_next, len(k_grid)):  
                y = y_grid[i, a]
                k = k_grid[j]
                c = y - k
                v_temp = (1 - β) * np.log(c) + β * E_v[j, a];

                if v_temp > v_max:
                    v_max = v_temp
                    k_prime = k_grid[j]
                    k_next = j
                else:
                    break 

            v_new[i, a] = v_max
            σ[i, a] = k_prime

    return v_new, σ

In [51]:
def solve_model(k_grid, 
                A, 
                π,
                α=1/3, 
                β=0.95,
                tol=1e-8,
                maxiter=1000):

    # Initialize values
    diff = 1e3
    i = 0
    v = np.zeros((len(k_grid), len(A)))

    while diff > tol and i < maxiter:       
            
        # Update value and policy functions for given productivity
        v_new, σ = innerloop(k_grid, v, A, π, β)

        diff = np.abs(v_new - v).max()
        v = v_new

        i += 1
        if(i % 20 == 0 or i == 1):
            print(f"Iteration {i}: diff = {diff}")
            
    print(f'Coverged in {i} iterations')
            
    return v_new, σ

In [53]:
%time solve_model(k_grid, A, π)

Iteration 1: diff = 0.05274159340733661
Iteration 20: diff = 0.018703461659313803
Iteration 40: diff = 0.006668550348659208
Iteration 60: diff = 0.0023813230210369962
Iteration 80: diff = 0.0008513486462362607
Iteration 100: diff = 0.00030462866646285836
Iteration 120: diff = 0.00010907234621893025
Iteration 140: diff = 3.907242347012119e-05
Iteration 160: diff = 1.4009761284849986e-05
Iteration 180: diff = 5.026883433956186e-06
Iteration 200: diff = 1.8037008289173428e-06
Iteration 220: diff = 6.471854268053079e-07
Iteration 240: diff = 2.3221595313316357e-07
Iteration 260: diff = 8.332106660180472e-08
Iteration 280: diff = 2.989628677951828e-08
Iteration 300: diff = 1.0727029597923377e-08
Coverged in 302 iterations
CPU times: user 3.68 s, sys: 327 ms, total: 4 s
Wall time: 1.49 s


(array([[-0.99728868, -0.98552368, -0.97409286, -0.9603671 , -0.95228169],
        [-0.99728731, -0.98552231, -0.97409149, -0.96036574, -0.95227998],
        [-0.99728594, -0.98552094, -0.97409012, -0.96036437, -0.95227828],
        ...,
        [-0.97049417, -0.95872916, -0.9472971 , -0.93356899, -0.9226772 ],
        [-0.97049371, -0.9587287 , -0.94729665, -0.93356853, -0.92267674],
        [-0.97049325, -0.95872825, -0.94729619, -0.93356808, -0.92267628]]),
 array([[0.13849505, 0.13996508, 0.14145011, 0.14299514, 0.2061863 ],
        [0.13849505, 0.13997008, 0.14145011, 0.14299514, 0.2061863 ],
        [0.13850006, 0.13997008, 0.14145511, 0.14300014, 0.2061863 ],
        ...,
        [0.19974118, 0.20186122, 0.20400626, 0.2061813 , 0.21277143],
        [0.19974118, 0.20186622, 0.20400626, 0.2061813 , 0.21277143],
        [0.19974118, 0.20186622, 0.20400626, 0.2061863 , 0.21277143]]))

Another claim of the paper is that "Julia benefits a lot from a mild investment on optimization and makes it easier to compare with `Numba` and `Cython`, which require extra work as well with respect to basic Python". The evidence they provide actually supports the opposite claim. After adapting their original code to Python 3, it turns out that only two lines need to be added (and a few of their inconsistencies deleted) to get `Numba` to run: `from numba import jit` and `@njit`. Not only is this extremely easy, but it is also much less work than AVF put into trying to optimize their Julia code, which can be partially observed by looking at the history of the file [here](https://github.com/jesusfv/Comparison-Programming-Languages-Economics/commits/master/RBC_Julia.jl). For timing purposes, we also remove the print statements, and add a warm-up period. We run the new code in the cell below.

## Julia version

To run the Julia code you will need to download it from [here](https://github.com/jstac/julia_python_comparison) and place it in the present working directory. We remove the print statements and add a warm-up iteration to be consistent with the Python code.

In [6]:
!julia RBC_Julia.jl

First run (warm-up): 
  3.992546 seconds (2.62 M allocations: 1.337 GiB, 7.72% gc time)
Second run: 
  2.335853 seconds (1.84 k allocations: 1.211 GiB, 3.89% gc time)


We are not the only ones who made similar observations. Other researchers report that Python is faster than Julia [here](https://discourse.julialang.org/t/a-comparison-of-programming-languages-in-economics/8966/19).